In [23]:

%load_ext autoreload 
# %aimport rl_envs.grid_world_env

%autoreload 2
import torch
import math
from torch.utils.tensorboard import SummaryWriter # type: ignore

from agents.policy_gradient_cliff import PGAgent
from tools.helper import *
import  gymnasium  as gym
from rl_envs.new_gym_grid_world_env import GridWorldEnv
from torch.nn import functional as F
from collections import defaultdict
import itertools
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
LEARN_RATE = 1e-2
DISCOUNTED_FACTOR = 0.99

FORBIDDEN_REWARD = -10
HITWALL_REWARD = -10
TARGET_REWARD = 1

SEED = 666

In [25]:
# env = GridWorldEnv(size=3,fixed_map = True, seed=SEED, forbidden_grids=[(1,1)], target_grids=[(2,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)
# env = GridWorldEnv(fixed_map = True, forbidden_grids=[(1,1),(1,2), (2,2),(3,1),(3,3),(4,1)], target_grids=[(3,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)

# env = gym.make("CliffWalking-v0")
# torch.autograd.set_detect_anomaly(False, check_nan=True)
# env.seed(args.seed)
torch.manual_seed(SEED)

In [26]:
episode_rewards = []
episode_lengths = []


In [28]:
env = gym.make("CliffWalking-v0")
agent = PGAgent(int(env.observation_space.n), int(env.action_space.n),  discounted_factor=1.)

# env = GridWorldEnv(fixed_map = True, forbidden_grids=[(1,1),(1,2), (2,2),(3,1),(3,3),(4,1)], target_grids=[(3,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)
# agent = PGAgent(2, 5, lr = LEARN_RATE, discounted_factor=DISCOUNTED_FACTOR)

# env = gym.make("CartPole-v1")
# agent = PGAgent(4, 2, lr = LEARN_RATE, discounted_factor=DISCOUNTED_FACTOR)

writer = SummaryWriter()
num_episodes = 600
episode_len = 10000
eps = np.finfo(np.float32).eps.item()
# 第一次收集改为随机收集
trajectory = []
obs, _ = env.reset()
# for _ in range(1000):
#     state = tuple(obs['agent'])
#     action = agent.get_behavior_action(state)
#     obs, reward, terminated, truncated, info = env.step(action)
#     trajectory.append((state, action, reward+10))
running_reward = -10
episode_rewards = defaultdict(float)
for i_episode  in range(num_episodes):
    # 首先, 根据 policy 生成 episode
    next_state, _ = env.reset()
    # trap_flag = False
    # 初始策略是不是有比较大的影响? 
    # del agent.saved_log_probs[:]
    for t in itertools.count():
        # del agent.saved_log_prob
        state = next_state
        action = agent.get_action(state) # action 这里也有随机性
        next_state, reward, terminated, truncated, info = env.step(action)

        # next_state process HERE

        # calculate TD target
        v_value = agent.value_net(state)
        value_next = agent.value_net(next_state)

        TD_target = reward + agent.discounted_factor * value_next
        TD_error = TD_target - v_value

        # Update statistics
        episode_rewards[i_episode] += reward

        agent.optimizer_v.zero_grad()
        loss2 =  F.mse_loss(TD_target, v_value)
        loss2.sum().backward(retain_graph=True)
        agent.optimizer_v.step()

        agent.optimizer.zero_grad()
        loss = -agent.saved_log_prob * TD_error
        loss.sum().backward()
        # torch.nn.utils.clip_grad.clip_grad_norm_(agent.policy_net.parameters(), 100)
        agent.optimizer.step()
        if t % 1000 == 0:
            print("\rStep {} @ Episode {}/{} ({}, {})".format(
                    t, i_episode + 1, num_episodes, episode_rewards[i_episode], episode_rewards[i_episode - 1]))

        if terminated or truncated:
            break

Step 0 @ Episode 1/600 (-1.0, 0.0)
Step 1000 @ Episode 1/600 (-3179.0, 0.0)
Step 2000 @ Episode 1/600 (-4575.0, 0.0)
Step 3000 @ Episode 1/600 (-5674.0, 0.0)
Step 4000 @ Episode 1/600 (-6773.0, 0.0)
Step 5000 @ Episode 1/600 (-7872.0, 0.0)
Step 6000 @ Episode 1/600 (-8872.0, 0.0)
Step 7000 @ Episode 1/600 (-9872.0, 0.0)
Step 8000 @ Episode 1/600 (-10872.0, 0.0)
Step 9000 @ Episode 1/600 (-11872.0, 0.0)
Step 10000 @ Episode 1/600 (-12872.0, 0.0)
Step 11000 @ Episode 1/600 (-13872.0, 0.0)
Step 12000 @ Episode 1/600 (-14872.0, 0.0)
Step 13000 @ Episode 1/600 (-15872.0, 0.0)
Step 14000 @ Episode 1/600 (-16872.0, 0.0)
Step 15000 @ Episode 1/600 (-17872.0, 0.0)
Step 16000 @ Episode 1/600 (-18872.0, 0.0)
Step 17000 @ Episode 1/600 (-19872.0, 0.0)
Step 18000 @ Episode 1/600 (-20872.0, 0.0)
Step 19000 @ Episode 1/600 (-21872.0, 0.0)
Step 20000 @ Episode 1/600 (-22872.0, 0.0)
Step 21000 @ Episode 1/600 (-23872.0, 0.0)
Step 22000 @ Episode 1/600 (-24872.0, 0.0)
Step 23000 @ Episode 1/600 (-25872.

KeyboardInterrupt: 

In [ ]:
# # visualize_in_gym(agent, "CartPole-v1")
# policy = agent.generate_policy_table(env.height, env.width)

# print_by_dict(env, policy)

# for i in range(env.height):
#     print("[", end=" ")
#     for j in range(env.width):
#         state = (i,j)
#         action = np.argmax(policy[state])
#         print(env.action_mappings[action], end=" ")
#     print("]")

In [ ]:
# env.max_steps = 10
# gridworld_demo(agent, env, repeat_times=500)
# gridworld_demo(agent, forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD)
# visualize_in_gym(agent, "CartPole-v1")
visualize_in_gym(agent, "CliffWalking-v0")


KeyboardInterrupt: 